In [23]:
import re
import torch
import numpy as np
import pandas as pd
from torch_geometric.data import Data

In [24]:
# read in txt file with extra info
# construct adjacency list
datapath = "scripts/parcel_neighbors.txt"
features = pd.read_csv(datapath, delimiter="\t", header=None)
features.columns = ['parcel', 'network', 'neighbors']

In [25]:
def interpret_neighbors(neighbors):
    temp = neighbors.split(" ")
    temp = [int(el) for el in temp if el != '']
    return temp

# formatting the list of neighboring regions
features['neighbors'] = features['neighbors'].apply(lambda x: interpret_neighbors(x))

In [26]:
# creating an edge list 
edge_col = []
for i,row in features.iterrows():
    neighbors = row['neighbors']
    edge_list = [[i+1, el] for el in neighbors]

    # converting to a tensor
    edge_tensor = torch.tensor(edge_list, dtype=torch.long).t().contiguous()
    edge_col.append(edge_tensor)
edge_col = pd.Series(edge_col)
features['edgeList_embeddings'] = edge_col
# features = pd.concat([features, edge_col], axis=1)

In [27]:
seven_networks = {
    'Vis':         0,
    'SomMot':      1,
    'DorsAttn':    2,
    'SalVentAttn': 3,
    'Limbic':      4,
    'Cont':        5,
    'Default':     6,

}

split_network_info = features['network'].apply(lambda x: x.split("_"))
network_embedding_col = []
for net in split_network_info:
    network_embedding = torch.zeros(8)
    net_idx = seven_networks[net[2]]
    network_embedding[net_idx] = 1
    if net[1] == 'LH':
        network_embedding[7] = 1
    network_embedding_col.append(network_embedding)
network_embedding_col = pd.Series(network_embedding_col)
features['network_embeddings'] = network_embedding_col
# features

In [43]:
test = features.iloc[0,:]
edge_embeddings = test['edgeList_embeddings'] 
network_embeddings = test['network_embeddings']

# network_embeddings_repeated = network_embeddings.unsqueeze(1).repeat(1, edge_embeddings.shape[1], 1)
network_embeddings_repeated = network_embeddings.unsqueeze(1).repeat(1, edge_embeddings.shape[1], 1)

combined_embeddings = torch.cat([edge_embeddings, network_embeddings_repeated], dim=2)


IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)

In [ ]:

ntwrk = test['network_embeddings'].unsqueeze(1)
ntwrk = ntwrk.expand(-1, 7)


test['network_embeddings']
# test['edgeList_embeddings'].shape[1]
torch.cat([test['edgeList_embeddings'], test['network_embeddings'].unsqueeze(1)], dim=1)

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 2 but got size 8 for tensor number 1 in the list.

In [32]:
test['edgeList_embeddings']

tensor([[  1,   1,   1,   1,   1,   1,   1],
        [  2,   3,   4,   7,  69, 123, 126]])

In [40]:
test['network_embeddings'].repeat

<function Tensor.repeat>